# Collect dependencies using `nbconvert`

The document creates an `nbconvert.preprocessor.execute` that will capture the current environment at the end of the execution.

In [7]:
    from pathlib import Path; import functools, io, nbconvert, ast, json, sys

In [8]:
    def get_imports_from_display(display): return display[0]['data']['text/plain']

In [8]:
    def import_to_env(str):
        from yaml import safe_load
        with io.StringIO(str) as s:
            return {'environment': {
                'dependencies': safe_load(s), 'channels': ['conda-forge', 'defaults'],}}

In [9]:
    mappings = dict(
        PIL='pillow',
        skimage='scikit-image', 
        yaml= 'pyyaml',
        delegator='delegator.py',
        pidgin="git+https://github.com/deathbeds/pidgin",
        importnb="git+https://github.com/deathbeds/importnb",
        depfinder="git+https://github.com/deathbeds/depfinder@ipython-code-transform",
        **{'cv2.cv2': 'opencv-python', });
    mappings.update(prompt_toolkit='prompt-toolkit==1.0.15')
    nah = 'sphinxcontrib', 'pkg_resources', 'dateutil', 'storemagic', 'OpenSSL', 'socks', 'gmpy2', 'blib2to3', 'deathbeds';
    special = "cv2.cv2",;

We desire to to find the current dependencies for a running kernel state.

> The last line will return the code in the current cell.

In [21]:
    def paths(): 
        from pandas import Series
        return Series(sys.path, sys.path).apply(
            lambda object: Series([
                relative_to(object, site)
                for site in __import__('site').getsitepackages()], __import__('site').getsitepackages())
        )[__import__('site').getsitepackages()[1:]].dropna()

In [22]:
    def relative_to(object, site):
        from toolz.curried import excepts
        return excepts(BaseException, lambda x: Path(x).relative_to(site),lambda e: None)(object)

In [23]:
    def modules(site_packages): 
        from pandas import Series
        return Series(sys.modules).pipe(
            lambda s: s[s.apply(lambda x: '.' not in x.__name__)]
        ).apply(
            lambda x: getattr(x, '__file__', None)
        ).dropna().apply(
            lambda x: Series([relative_to(x, object) for object in site_packages.index])
        ).dropna(how='all').pipe(lambda df: df[df.index.map(lambda x: not x.startswith('_') and ('.' not in x))])

In [24]:
    def requirements():
        import sys, copy
        requirements = ""
        sys_modules = copy.copy(sys.modules)
        from pandas import Series
        from toolz.curried import excepts
        
        site_packages = paths()

        packages_names = modules(paths())
        return packages_names.index.tolist()

In [25]:
    class RequiresPreprocessor(nbconvert.preprocessors.execute.ExecutePreprocessor):
        def preprocess_cell(self, cell, resources, cell_index):
            from nbformat.v4 import new_code_cell
            cell, resources = super().preprocess_cell(cell, resources, cell_index)
            if cell_index == len(self.nb.cells)-1:
                reply, display = self.run_cell(new_code_cell(
                    """__import__('importlib').import_module('deathbeds.2018_07_29_Execute_and_discover_imports').requirements()"""
                ))
                self.nb['metadata'].update(
                    import_to_env(get_imports_from_display(display))
                )
                self.nb['metadata']['environment'].update(name=resources['metadata'].get('name', 'sure'))
                print(resources)
            return cell, resources

In [26]:
    def _test_exporter():
        from nbconvert.exporters import NotebookExporter; import json
        exporter = NotebookExporter(preprocessors=[RequiresPreprocessor(enabled=True)])
    
        nb, _ = exporter.from_filename("2018-07-29-Execute-and-discover-imports.ipynb")
    
        assert json.loads(nb)['metadata']['environment']['dependencies']

The default config file name for nbconvert is `jupyter_nbconvert_config.py`.

    %%file jupyter_nbconvert_config.py
    with __import__('importnb').Notebook():
        from deathbeds.__Execute_and_discover_imports import RequiresPreprocessor
    c.Exporter.preprocessors = [RequiresPreprocessor()]

    !jupyter nbconvert --to notebook 2018-07-29-Execute-and-discover-imports.ipynb